### Impact of Embeddings on Quality with Sub Question Query

In this tutorial, we load longer text (Alice in Wonderland) and utilize Llama-Index Sub Question Query to evlauate a complex questions on the book.

In addition, we will iterate through different embeddings and chunk sizes and use TruLens to select the best one.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/frameworks/llama_index/llamaindex-subquestion-query.ipynb)

In [ ]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes. 
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.  
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "..."
os.environ["HUGGINGFACE_API_KEY"] = "..."

In [ ]:
# Import main tools for building app
from llama_index import VectorStoreIndex, SimpleWebPageReader, ServiceContext
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine

# load data


documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["https://www.gutenberg.org/files/11/11-h/11-h.htm"]
)

In [ ]:
# Imports main tools for eval
from trulens_eval import TruLlama, Feedback, Tru, feedback
tru = Tru()

#hugs = feedback.Huggingface()
openai = feedback.OpenAI()

# Question/answer relevance between overall question and answer.
model_agreement = Feedback(openai.model_agreement).on_input_output()

In [ ]:
tru.run_dashboard()

In [ ]:
# iterate through embeddings and chunk sizes, evaluating each response's agreement with chatgpt using TruLens
embeddings = ['text-embedding-ada-001','text-embedding-ada-002']
query_engine_types = ['VectorStoreIndex','SubQuestionQueryEngine']

service_context=512

for embedding in(embeddings):
    for query_engine_type in query_engine_types:

            # build index and query engine
            index = VectorStoreIndex.from_documents(documents)

            # create embedding-based query engine from index
            query_engine = index.as_query_engine(embed_model=embedding)

            if query_engine_type == 'SubQuestionQueryEngine':
                service_context = ServiceContext.from_defaults(chunk_size=512)
                # setup base query engine as tool
                query_engine_tools = [
                    QueryEngineTool(
                        query_engine=query_engine,
                        metadata=ToolMetadata(name='Alice in Wonderland', description='THE MILLENNIUM FULCRUM EDITION 3.0')
                    )
                ]
                query_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools, service_context=service_context)
            else:
                pass         

            tc = TruLlama(app_id = f'{query_engine_type}_{embedding}', app = query_engine, feedbacks = [model_agreement])

            response = tc.query("Describe Alice's growth from meeting the White Rabbit to challenging the Queen of Hearts?")
            response = tc.query("Relate aspects of enchantment to the nostalgia that Alice experiences in Wonderland. Why is Alice both fascinated and frustrated by her encounters below-ground?")
            response = tc.query("Describe the White Rabbit's function in Alice.")
            response = tc.query("Describe some of the ways that Carroll achieves humor at Alice's expense.")
            response = tc.query("Compare the Duchess' lullaby to the 'You Are Old, Father William' verse")
            response = tc.query("Compare the sentiment of the Mouse's long tale, the Mock Turtle's story and the Lobster-Quadrille.")
            response = tc.query("Summarize the role of the mad hatter in Alice's journey")
            response = tc.query("How does the Mad Hatter influence the arc of the story throughout?")
